In [1]:
# solutions should be to send bob a smooth prime then pohlig-hellman, but it's screwed up for some reason

In [3]:
from Crypto.Util.number import getPrime
from sympy import isprime

def get_smooth_prime(bits, smoothness=16):
    p = 2
    p_factors = [p]
    while p.bit_length() < bits - 2 * smoothness:
        factor = getPrime(smoothness)
        p_factors.append(factor)
        p *= factor
    bitcnt = (bits - p.bit_length()) // 2
    while True:
        prime1 = getPrime(bitcnt)
        prime2 = getPrime(bitcnt)
        tmpp = p * prime1 * prime2
        if tmpp.bit_length() < bits:
            bitcnt += 1
            continue
        if tmpp.bit_length() > bits:
            bitcnt -= 1
            continue
        if isprime(tmpp + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = tmpp + 1
            break
    p_factors.sort()
    return (p, p_factors)
sprime, sprime_factors = get_smooth_prime(1536)

In [6]:
from sympy import factorint
factorint(sprime-1, use_ecm=True)

In [55]:
# only found accepted value of g is 2 so far
# A will be sussed out for [0, 2]. this probably doesn't matter
# p will be sussed out if not 1536 bits
# doesn't check for strong prime, we might be able to discrete log 
from pwnlib.tubes.remote import remote
from cryptolib import json_send, diffie_hellman_decrypt_flag, json_recv
from json import loads, dumps
from sympy import prime
r = remote("socket.cryptohack.org", 13378)
r.recvuntil(b"Intercepted from Alice: ")
alice_public = json_recv(r)
p, g, A = int(alice_public['p'], 16), int(alice_public['g'], 16), int(alice_public['A'], 16)
r.recvuntil(b"Intercepted from Bob: ")
B = int(json_recv(r)['B'], 16)
r.recvuntil(b"Intercepted from Alice: ")
alice_aes = json_recv(r)
iv, encrypted = alice_aes['iv'], alice_aes['encrypted']
r.recvuntil(b"Bob connects to you, send him some parameters: ")


json_send(r, {"p": hex(sprime), "g": hex(2), "A": hex(3)})
r.recvuntil(b"Bob says to you: ")
B_ = int(loads(r.recvline())['B'], 16)

In [56]:
sprime, B_, 2

(1397191800548672273568811996651742154191247479714896585513003309885441250487231630040942454320158123044330563607720122363149183347498551928718603442814846444437804352192504271370337117861035540279811659227999194289902689066809261485239889246551167615099051162940583754544791726318799367291958635470164923966165180677461889299939938495590918934546584319339760887931048148085333046640569211751659253603387231170016532360253260717696789625016049027595815278647266923,
 326188563042740220916025325949472061600277512079685103313937929105348103956642328397038734619630095372348231109219291249100511859902205119647150405170810628266218072469696281495314846771121546210871262963057080188901983629929922647874148411794673091889452123714358654485615939464958457784414407324602049778825333277432440638592360936646380761007167920865432211481859380963588020227097928685920294134213453872770860387438288442072732267262002912667862492381307464,
 2)

In [23]:
from sympy.ntheory.residue_ntheory import _discrete_log_pohlig_hellman
_discrete_log_pohlig_hellman(n,a,b,order,order_factors)

0

In [63]:
from Crypto.Util.number import inverse, getPrime
getPrime(1536)

1654870807489741666166712504001835292191126761314153027652346620421649434971976149797565316194747539944117759592727080300884221376989401231040881816893012817556464231807564159340373964933698451719001929968306564707001329275843374696183366614294595402193412758168750368154847505511192419166115921252541617998733354198358398654112905044050386599728877889736600813889719539656152096140540920070332687041208010905095616591952709825797184093420086492088130092808262803

b'B'

In [4]:
p, g, A

(2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919,
 2,
 1842456548509985448001501955708317543923668882702387731149842197334091656598019912729386918817972088534618034054199283446233163815232643357049841287518070880509964732159087089881097780930507595659799099021074417551186847518515681575923228780734810350836185507503936008727914189692134425269770264193394331644892088233122500403697047904867774929619033647804401486714172117981128849420027222802210971219407887958527960013507993788267466267953204739503158129814352528)

In [5]:
B

714216756598305735603046246561487555788511628802563305910135524591202127744114943901795325807949250885402626819853303127965411915027566384701087862456809063646195165193592925563073929195170506328048913324457052878973166150910160579628348050428532539130082307794315984411600494511723848533327684422477621927785771984975971767068564356745941560374525498015524036641953306600952157983344056291242132311780666645344225431766551347466472617080772291014016878534899147